Imports and functions

In [683]:
import os
import netCDF4 as nc4
import numpy as np
#import numpy.ma as ma

User inputs

In [684]:
indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/06/test'
#indir = '/home/disk/bob/gpm/nam_ku/classify/ex_data/2014/11/test'
THRESHOLD = 38.0  #dBZ
SLOPE_LO = 6.0
SLOPE_HI = 8.0
VERT_RES = 0.125  #km

In [685]:
#for fname in os.listdir(indir):
#   print fname

In [686]:
    fname = 'GPM-2Ku.001497.20140603.213419.nc'
    # Test cases for 201411
    #fname = 'GPM-2Ku.004022.20141113.044926.nc'
    #fname = 'GPM-2Ku.004186.20141123.171346.nc'
    #fname = 'GPM-2Ku.004201.20141124.162249.nc'

In [687]:
    #ncid = nc4.Dataset(indir+'/'+fname,'a')
    ncid = nc4.Dataset(indir+'/'+fname,'r')
    
    refl = np.array(ncid.variables['refl'])
    (ntime,nalt,nlat,nlon) = refl.shape
    refl_missing = ncid.variables['refl'].missing_value
    bin_missing = int(refl_missing)
    
    rt = np.array(ncid.variables['rain_type'])
    rt_missing = ncid.variables['rain_type'].missing_value
    #rt[(rt==rt_missing)] = np.nan 
    
    rt_uw2 = np.array(rt, copy=True)    
    rt_uw3 = np.array(rt, copy=True)    

    For rt_uw2, change rain_type to stratiform in rt_uw if:
    1. rain_type = convective and
    2. max_refl in column >= THRESHOLD and
    3. (max_refl - dbz@(max_refl_ht-1km)/deltaHt <= abs(SLOPE_LO)
    4. (max_refl - dbz@(max_refl_ht+1km)/deltaHt >= abs(SLOPE_HI)
    
    For rt_uw3, change rain_type to stratiform in rt_uw if:
    1. rain_type = convective and
    2. max_refl in column >= THRESHOLD and
    3. (max_refl - dbz@(max_refl_ht+1km)/deltaHt >= abs(SLOPE_HI)    

In [688]:
    # find max_refl and max_refl_ht at each lat/lon
    max_refl = np.zeros(rt.shape,dtype=float)+refl_missing
    max_refl = np.amax(refl,axis=1)
    max_refl_bin = np.zeros(rt.shape,dtype=int)+bin_missing
    max_refl_bin = np.argmax(refl,axis=1)
    max_refl_ht = max_refl_bin * VERT_RES

    max_refl_bin[(max_refl==refl_missing)] = bin_missing
    max_refl_ht[(max_refl==refl_missing)] = refl_missing
    
    # find bin & height of highest good refl at each lat/lon
    max_ht_bin = np.zeros(rt.shape,dtype=int) + bin_missing
    for lev in range(nalt-1,-1,-1):
        max_ht_bin = np.where(np.logical_and((refl[:,lev,:,:] != refl_missing),
                                             (max_ht_bin == bin_missing) ), 
                            lev, max_ht_bin ) 
    max_ht = max_ht_bin * (max_ht_bin != bin_missing) * VERT_RES
    max_ht[(max_ht_bin==bin_missing)] = refl_missing    
    
    # find bin & height of lowest good refl at each lat/lon
    min_ht_bin = np.zeros(rt.shape,dtype=int) + bin_missing
    for lev in range(0,nalt):
        min_ht_bin = np.where(np.logical_and( (refl[:,lev,:,:] != refl_missing),
                                         (min_ht_bin == bin_missing) ), 
                            lev, min_ht_bin ) 
    min_ht = min_ht_bin * (min_ht_bin != bin_missing) * VERT_RES
    min_ht[(min_ht_bin==bin_missing)] = refl_missing    

In [689]:
    # Determine indices of interest 
    change = np.logical_and(rt==2,max_refl>=THRESHOLD)
    ind_time,ind_lat,ind_lon = np.where(change==True)
    npixels = ind_time.size

In [690]:
        # Run test for rt_uw2 and rt_uw3
        #for ind in range(0,npixels):
        #itime = ind_time[ind]
        #ilat = ind_lat[ind]
        #ilon = ind_lon[ind]
        itime = 0
        # Problem point with max refl at 0.125km
        #ilat = 266
        #ilon =  24
        # Good point with max refl at 1.75km
        #ilat = 265
        #ilon =  34
        # Fails at SLOPE_HI=8, passes at SLOPE_HI=6
        #ilat = 264
        #ilon = 25
        ilon = 100
        ilat = 45
        print 'ind = ',ind,' and [itime,ilat,ilon] = [',itime,',',ilat,',',ilon,'] and max_refl = ',max_refl[itime,ilat,ilon]
        if max_refl[itime,ilat,ilon] != refl_missing:
            print 'max_refl_bin = ',max_refl_bin[itime,ilat,ilon],' and max_refl = ',max_refl[itime,ilat,ilon]
            print 'Looking for minus point'
            if max_refl_bin[itime,ilat,ilon] >= int(1.0/VERT_RES):
                print '   Not atypical'
                ht_index = max_refl_bin[itime,ilat,ilon]-int(1.0/VERT_RES)
                print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
                refl_minus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
                ht_minus_diff = 1.0
                print '   refl_minus_diff = ',refl_minus_diff,' and ht_minus_diff = ',ht_minus_diff
            else:
                print '   Atypical'
                ht_index = min_ht_bin[itime,ilat,ilon]
                print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
                refl_minus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
                ht_minus_diff = max_refl_ht[itime,ilat,ilon]-min_ht[itime,ilat,ilon]
                print '   refl_minus_diff = ',refl_minus_diff,' and ht_minus_diff = ',ht_minus_diff
            if ht_minus_diff != 0.0:
                slope_minus = refl_minus_diff/ht_minus_diff
            else:
                slope_minus = refl_missing
            print '   slope_minus = ',slope_minus
        
            print 'Looking for plus point'
            if max_ht_bin[itime,ilat,ilon] >= max_refl_bin[itime,ilat,ilon]+int(1.0/VERT_RES):
                print '   Not atypical'
                ht_index = max_refl_bin[itime,ilat,ilon]+int(1.0/VERT_RES)
                print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
                refl_plus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
                ht_plus_diff = 1.0
                print '   refl_plus_diff = ',refl_plus_diff,' and ht_plus_diff = ',ht_plus_diff
            else:
                print '   Atypical'
                ht_index = max_ht_bin[itime,ilat,ilon]
                print '   ht_index = ',ht_index,' and refl = ',refl[itime,ht_index,ilat,ilon]
                refl_plus_diff = max_refl[itime,ilat,ilon]-refl[itime,ht_index,ilat,ilon]
                print 'refl = ',
                ht_plus_diff = max_ht[itime,ilat,ilon]-max_refl_ht[itime,ilat,ilon]
                print '   refl_plus_diff = ',refl_plus_diff,' and ht_plus_diff = ',ht_plus_diff
            if ht_plus_diff != 0.0:
                slope_plus = refl_plus_diff/ht_plus_diff
            else:
                slope_plus = refl_missing
            print '   slope_plus = ',slope_plus
                
            change2 = np.logical_and(abs(slope_minus) <= SLOPE_LO,
                                     abs(slope_plus) >= SLOPE_HI)
            print 'change2 = ',change2
            if change2 == True:
                rt_uw2[itime,ilat,ilon] = 1
                
            change3 = abs(slope_plus) >= SLOPE_HI
            print 'change3 = ',change3
            if change3 == True:
                rt_uw3[itime,ilat,ilon] = 1
                       

ind =  689  and [itime,ilat,ilon] = [ 0 , 45 , 100 ] and max_refl =  46.26
max_refl_bin =  23  and max_refl =  46.26
Looking for minus point
   Not atypical
   ht_index =  15  and refl =  45.3
   refl_minus_diff =  0.959999  and ht_minus_diff =  1.0
   slope_minus =  0.959999084473
Looking for plus point
   Not atypical
   ht_index =  31  and refl =  42.9
   refl_plus_diff =  3.36  and ht_plus_diff =  1.0
   slope_plus =  3.35999679565
change2 =  False
change3 =  False


In [691]:
refl[0,:,45,100]

array([-99.        , -99.        , -99.        ,  45.29000092,
        45.29000092,  45.29999924,  45.29999924,  45.29999924,
        45.29999924,  45.29999924,  45.29000092,  45.29999924,
        45.29999924,  45.29999924,  45.29000092,  45.29999924,
        45.29999924,  45.29999924,  45.29999924,  45.84999847,
        46.02999878,  45.16999817,  45.72999954,  46.25999832,
        45.40999985,  44.24000168,  45.15000153,  44.68999863,
        43.86999893,  42.04999924,  42.31000137,  42.90000153,
        40.75999832,  39.29999924,  37.16999817,  36.43999863,
        34.29999924,  32.84000015,  32.09999847,  30.62999916,
        30.62000084,  29.87000084,  29.86000061,  28.71999931,
        27.95000076,  29.06999969,  29.43000031,  29.04000092,
        29.04000092,  27.5       ,  28.25      ,  27.46999931,
        26.27000046,  25.45000076,  25.        ,  24.98999977,
        24.12000084,  24.11000061,  22.21999931,  21.71999931,
        20.04000092,  19.39999962,  18.02000046,  18.75

    Add new raintype fields to input netcdf file

In [681]:
    rt_uw2_id = ncid.createVariable('rain_type_uw2','f4',
                                   ('time','latitude','longitude'),zlib=True)
    #rt_uw2_id = ncid.createVariable('rain_type_uw2','f4',
    #                              ('time','latitude','longitude'))
    rt_uw2_id.units = "none"
    rt_uw2_id.long_name = "Rain Type UW2"
    rt_uw2_id.stratiform = 1
    rt_uw2_id.convective = 2
    rt_uw2_id.other = 3
    rt_uw2_id.missing_value = rt_missing
    rt_uw2_id[:,:,:] = rt_uw2
    
    rt_uw3_id = ncid.createVariable('rain_type_uw3','f4',
                                   ('time','latitude','longitude'),zlib=True)
    #rt_uw3_id = ncid.createVariable('rain_type_uw3','f4',
    #                              ('time','latitude','longitude'))
    rt_uw3_id.units = "none"
    rt_uw3_id.long_name = "Rain Type UW3"
    rt_uw3_id.stratiform = 1
    rt_uw3_id.convective = 2
    rt_uw3_id.other = 3
    rt_uw3_id.missing_value = rt_missing
    rt_uw3_id[:,:,:] = rt_uw3

In [682]:
    ncid.close()